<a href="https://colab.research.google.com/github/ranwiththecode/fantasy_lit_sent_an/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import re
import pandas as pd
from google.colab import drive
from tqdm import tqdm

# Mount Google Drive
drive.mount('/content/drive')

# Comprehensive emoji dictionary
EMOJI_DICT = {
    # Smileys & Emotion
    "😀": "[grinning_face]", "😃": "[grinning_face_with_big_eyes]", "😄": "[grinning_face_with_smiling_eyes]",
    "😁": "[beaming_face_with_smiling_eyes]", "😆": "[grinning_squinting_face]", "😅": "[grinning_face_with_sweat]",
    "😂": "[face_with_tears_of_joy]", "🤣": "[rolling_on_the_floor_laughing]", "😊": "[smiling_face_with_smiling_eyes]",
    "😇": "[smiling_face_with_halo]", "🙂": "[slightly_smiling_face]", "🙃": "[upside_down_face]",
    "😉": "[winking_face]", "😌": "[relieved_face]", "😍": "[smiling_face_with_heart_eyes]",
    "🥰": "[smiling_face_with_hearts]", "😘": "[face_blowing_a_kiss]", "😗": "[kissing_face]",
    "😙": "[kissing_face_with_smiling_eyes]", "😚": "[kissing_face_with_closed_eyes]", "😋": "[face_savoring_food]",
    "😛": "[face_with_tongue]", "😝": "[squinting_face_with_tongue]", "😜": "[winking_face_with_tongue]",
    "🤪": "[zany_face]", "🤨": "[face_with_raised_eyebrow]", "🧐": "[face_with_monocle]",
    "🤓": "[nerd_face]", "😎": "[smiling_face_with_sunglasses]", "🤩": "[star_struck]",
    "🥳": "[partying_face]", "😏": "[smirking_face]", "😒": "[unamused_face]",
    "😞": "[disappointed_face]", "😔": "[pensive_face]", "😟": "[worried_face]",
    "😕": "[confused_face]", "🙁": "[slightly_frowning_face]", "☹️": "[frowning_face]",
    "😣": "[persevering_face]", "😖": "[confounded_face]", "😫": "[tired_face]",
    "😩": "[weary_face]", "🥺": "[pleading_face]", "😢": "[crying_face]",
    "😭": "[loudly_crying_face]", "😤": "[face_with_steam_from_nose]", "😠": "[angry_face]",
    "😡": "[pouting_face]", "🤬": "[face_with_symbols_on_mouth]", "🤯": "[exploding_head]",
    "😳": "[flushed_face]", "🥵": "[hot_face]", "🥶": "[cold_face]",
    "😱": "[face_screaming_in_fear]", "😨": "[fearful_face]", "😰": "[anxious_face_with_sweat]",
    "😥": "[sad_but_relieved_face]", "😓": "[downcast_face_with_sweat]", "🤗": "[hugging_face]",
    "🤔": "[thinking_face]", "🤭": "[face_with_hand_over_mouth]", "🤫": "[shushing_face]",
    "🤥": "[lying_face]", "😶": "[face_without_mouth]", "😐": "[neutral_face]",
    "😑": "[expressionless_face]", "😬": "[grimacing_face]", "🙄": "[face_with_rolling_eyes]",
    "😯": "[hushed_face]", "😦": "[frowning_face_with_open_mouth]", "😧": "[anguished_face]",
    "😮": "[face_with_open_mouth]", "😲": "[astonished_face]", "🥱": "[yawning_face]",
    "😴": "[sleeping_face]", "🤤": "[drooling_face]", "😪": "[sleepy_face]",
    "😵": "[dizzy_face]", "🤐": "[zipper_mouth_face]", "🥴": "[woozy_face]",
    "🤢": "[nauseated_face]", "🤮": "[face_vomiting]", "🤧": "[sneezing_face]",
    "😷": "[face_with_medical_mask]", "🤒": "[face_with_thermometer]", "🤕": "[face_with_head_bandage]",

    # Hearts
    "❤️": "[red_heart]", "🧡": "[orange_heart]", "💛": "[yellow_heart]",
    "💚": "[green_heart]", "💙": "[blue_heart]", "💜": "[purple_heart]",
    "🖤": "[black_heart]", "🤍": "[white_heart]", "🤎": "[brown_heart]",
    "💔": "[broken_heart]", "❤️‍🔥": "[heart_on_fire]", "❤️‍🩹": "[mending_heart]",
    "💕": "[two_hearts]", "💞": "[revolving_hearts]", "💓": "[beating_heart]",
    "💗": "[growing_heart]", "💖": "[sparkling_heart]", "💘": "[heart_with_arrow]",
    "💝": "[heart_with_ribbon]", "💟": "[heart_decoration]",

    # Common symbols
    "⭐": "[star]", "🌟": "[glowing_star]", "✨": "[sparkles]",
    "💫": "[dizzy]", "🔥": "[fire]", "💯": "[hundred_points]",
    "💢": "[anger_symbol]", "💥": "[collision]", "💦": "[sweat_droplets]",
    "💨": "[dashing_away]", "💣": "[bomb]", "💬": "[speech_balloon]",
    "👁️‍🗨️": "[eye_in_speech_bubble]", "🗨️": "[left_speech_bubble]", "🗯️": "[right_anger_bubble]",
    "💭": "[thought_balloon]", "💤": "[zzz]",

    # Books & writing
    "📚": "[books]", "📖": "[open_book]", "📕": "[closed_book]",
    "📘": "[blue_book]", "📙": "[orange_book]", "📗": "[green_book]",
    "📓": "[notebook]", "📔": "[notebook_with_decorative_cover]", "📒": "[ledger]",
    "📑": "[bookmark_tabs]", "📝": "[memo]", "💻": "[laptop]",

    # Hands
    "👍": "[thumbs_up]", "👎": "[thumbs_down]", "✊": "[raised_fist]",
    "👊": "[oncoming_fist]", "🤛": "[left_facing_fist]", "🤜": "[right_facing_fist]",
    "👏": "[clapping_hands]", "🙌": "[raising_hands]", "👐": "[open_hands]",
    "🤲": "[palms_up_together]", "🤝": "[handshake]", "🙏": "[folded_hands]",
}

def preprocess_text(text):
    """Clean text by removing HTML tags, normalizing emojis, and standardizing whitespace"""
    if not isinstance(text, str):
        return ""

    # Remove HTML tags
    clean_text = re.sub(r'<[^>]+>', '', text)

    # Convert emojis to text descriptions
    for emoji, desc in EMOJI_DICT.items():
        clean_text = clean_text.replace(emoji, f" {desc} ")

    # Standardize whitespace and clean up
    clean_text = re.sub(r'\s+', ' ', clean_text).strip()

    return clean_text

def is_english(text, min_english_chars=0.7):
    """Simple English detection using character analysis"""
    if not isinstance(text, str) or not text.strip():
        return False

    # Count ASCII letters and basic punctuation
    english_chars = sum(1 for c in text if ord(c) < 128 or c in '‘’“”–—')
    total_chars = max(1, len(text))  # avoid division by zero

    return (english_chars / total_chars) >= min_english_chars

def preprocess_reviews_file(input_path, output_path):
    """Process the entire CSV file with progress tracking and validation"""
    try:
        # Read with error handling
        df = pd.read_csv(input_path, keep_default_na=False)
    except Exception as e:
        print(f"Error reading file: {e}")
        return False

    # Validate input structure
    if 'text' not in df.columns or 'rating' not in df.columns:
        print("Error: Input file must contain 'text' and 'rating' columns")
        return False

    # Convert 0 ratings to NULL (Goodreads uses 0 for "no rating")
    df['rating'] = df['rating'].replace(0, pd.NA)

    # Initialize progress bar for preprocessing
    tqdm.pandas(desc="Cleaning text")
    df['clean_text'] = df['text'].progress_apply(preprocess_text)

    # Filter English reviews with progress
    tqdm.pandas(desc="Checking language")
    df['is_english'] = df['clean_text'].progress_apply(is_english)
    english_df = df[df['is_english']].copy()

    # Keep only needed columns
    english_df = english_df[['rating', 'clean_text']]
    english_df.columns = ['rating', 'text']  # Standardize column names

    # Save with error handling
    try:
        english_df.to_csv(output_path, index=False, encoding='utf-8', na_rep='NULL')
        print(f"\nSuccess! Cleaned {len(english_df)}/{len(df)} reviews saved to: {output_path}")
        print(f"Reviews with valid ratings: {english_df['rating'].notna().sum()}")
        print(f"Reviews without ratings: {english_df['rating'].isna().sum()}")

        # Show sample of cleaned data
        print("\nSample of cleaned reviews:")
        print(english_df.sample(min(5, len(english_df))).to_string(index=False))
        return True
    except Exception as e:
        print(f"Error saving file: {e}")
        return False

# Example usage in Colab
if __name__ == "__main__":
    # Set your file paths
    BOOK_TITLE = "eragon_paolini"  # Change to your book title
    INPUT_PATH = f'/content/drive/MyDrive/Goodreads_Data/{BOOK_TITLE}_reviews.csv'
    OUTPUT_PATH = f'/content/drive/MyDrive/Goodreads_Data/{BOOK_TITLE}_clean.csv'

    # Run preprocessing
    print(f"Starting preprocessing for {BOOK_TITLE}...")
    success = preprocess_reviews_file(INPUT_PATH, OUTPUT_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Starting preprocessing for eragon_paolini...


Checking language: 100%|██████████| 31250/31250 [00:00<00:00, 33746.96it/s]



Success! Cleaned 31116/31250 reviews saved to: /content/drive/MyDrive/Goodreads_Data/eragon_paolini_clean.csv
Reviews with valid ratings: 29622
Reviews without ratings: 1494

Sample of cleaned reviews:
rating                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       